In [179]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geoip2.database
import myLogReader as mlr
import re

%matplotlib inline

In [149]:
reader = geoip2.database.Reader('../data/GeoLite2-City_20181009/GeoLite2-City.mmdb')
#reader.close()


In [150]:
iis_logs_df = pd.read_csv("../data/logs/u_ex181010.log"
            #,skiprows=[0,1,2,3]
            , comment='#'
            , sep=' ' 
            , usecols=[0,1, 2, 5, 6, 7, 8, 9, 10,11,12,14]
            , na_values='-'
            , names=['date'
                    ,'time'
                    ,'server-ip'
                    ,'cs-uri-query'
                    ,'server-port'
                    ,'cs-username'
                    ,'client-ip'
                    ,'cs(User-Agent)'
                    ,'cs(Referer)'
                    ,'sc-status'
                    ,'sc-substatus'
                   ,'time-taken(ms)'])

In [151]:
iis_logs_df.head()

,date,time,server-ip,cs-uri-query,server-port,cs-username,client-ip,cs(User-Agent),cs(Referer),sc-status,sc-substatus,time-taken(ms)
0,2018-10-10,00:00:19,192.168.2.210,NaN,80,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,301,0,93
1,2018-10-10,00:00:19,192.168.2.210,NaN,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,302,0,93
2,2018-10-10,00:00:19,192.168.2.210,ReturnUrl=%2f,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,302,0,93
3,2018-10-10,00:00:19,192.168.2.210,ReturnUrl=%2f,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,200,0,171
4,2018-10-10,00:00:20,192.168.2.210,v=pIhLJRZ-i9xD3brYZ6I5pZD6973WqBDt6kYnAg5j20E1,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,https://inplace.canberra.edu.au/Security/Accou...,200,0,265


In [152]:
iis_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12334 entries, 0 to 12333
Data columns (total 12 columns):
date              12334 non-null object
time              12334 non-null object
server-ip         12334 non-null object
cs-uri-query      5510 non-null object
server-port       12334 non-null int64
cs-username       7275 non-null object
client-ip         12334 non-null object
cs(User-Agent)    12138 non-null object
cs(Referer)       9206 non-null object
sc-status         12334 non-null int64
sc-substatus      12334 non-null int64
time-taken(ms)    12334 non-null int64
dtypes: int64(4), object(8)
memory usage: 1.1+ MB


In [153]:
iis_logs_df.describe()

,server-port,sc-status,sc-substatus,time-taken(ms)
count,12334.000000,12334.000000,12334.0,12334.000000
mean,442.323091,215.232447,0.0,532.415518
std,15.661410,38.194092,0.0,2264.506444
min,80.000000,200.000000,0.0,0.000000
25%,443.000000,200.000000,0.0,62.000000
50%,443.000000,200.000000,0.0,187.000000
75%,443.000000,200.000000,0.0,330.000000
max,443.000000,404.000000,0.0,82662.000000


In [158]:
iis_logs_df["server-port"].describe()

count    12334.000000
mean       442.323091
std         15.661410
min         80.000000
25%        443.000000
50%        443.000000
75%        443.000000
max        443.000000
Name: server-port, dtype: float64

In [166]:
#Enhance dataset with Client city and country , derived from IP
iis_logs_df['client-city'] =  iis_logs_df['client-ip'].apply(lambda x: reader.city(ip_address=x).city.name if reader.city(ip_address=x).city.name != None else np.nan)
iis_logs_df['client-country'] =  iis_logs_df['client-ip'].apply(lambda x: reader.city(ip_address=x).country.name if reader.city(ip_address=x).country.name != None else np.nan)

In [167]:
iis_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12334 entries, 0 to 12333
Data columns (total 16 columns):
date              12334 non-null object
time              12334 non-null object
server-ip         12334 non-null object
cs-uri-query      5510 non-null object
server-port       12334 non-null int64
cs-username       7275 non-null object
client-ip         12334 non-null object
cs(User-Agent)    12138 non-null object
cs(Referer)       9206 non-null object
sc-status         12334 non-null int64
sc-substatus      12334 non-null int64
time-taken(ms)    12334 non-null int64
client-city       12208 non-null object
client-country    12334 non-null object
client-device     12334 non-null object
client-browser    12334 non-null object
dtypes: int64(4), object(12)
memory usage: 1.5+ MB


In [227]:
## Define method to derive the type of Device used to access the website (Mobile or Desktop)

def getDevice (UserAgentResponse):
    device ='Other' 
    if 'Mobi' in UserAgentResponse:
        device = 'Mobile'
    else:
        device = 'Desktop'
    return device

def getBrowser (UserAgentResponse):
    browser ='Other' 
    if 'Firefox' in UserAgentResponse and 'Seamonkey' not in UserAgentResponse:
        browser = 'Firefox'
    elif 'Seamonkey' in UserAgentResponse:
        browser = 'Seamonkey'
    elif 'Chrome' in UserAgentResponse and 'Chromium' not in UserAgentResponse:
        browser = 'Chrome'
    elif ('Safari' in UserAgentResponse and 'Chromium' not in UserAgentResponse and 'Chrome' not in UserAgentResponse):
        browser = 'Safari'
    elif 'OPR' in UserAgentResponse and 'Opera'  in UserAgentResponse:
        browser = 'Opera'
    elif '; MSIE' in UserAgentResponse:
        browser = 'IE'
    return browser

def GetWebPageSection(x):
    section = 'Unknown'
    r = re.compile('([A-Z])\w+')
    section = r.search(x)
    if section is not None:
        return section.group()
    return section

iis_logs_df['client-device'] =  iis_logs_df['cs(User-Agent)'].apply(lambda x: getDevice(str(x)))

iis_logs_df['client-browser'] =  iis_logs_df['cs(User-Agent)'].apply(lambda x: getBrowser(str(x)))

iis_logs_df['client-webPage'] = iis_logs_df['cs(Referer)'].apply(lambda x: GetWebPageSection(x) if type(x) != float else np.nan)

In [226]:
iis_logs_df.loc[:, iis_logs_df.dtypes == object].describe()

,date,time,server-ip,cs-uri-query,cs-username,client-ip,cs(User-Agent),cs(Referer),client-city,client-country,client-device,client-browser,client-webPage
count,12334,12334,12334,5510,7275,12334,12138,9206,12208,12334,12334,12334,7855
unique,1,6220,1,2716,84,222,90,313,66,14,2,4,17
top,2018-10-10,02:56:42,192.168.2.210,ReturnUrl=%2f,s437875,137.92.20.64,Mozilla/5.0+(Windows+NT+10.0;+WOW64)+AppleWebK...,https://inplace.canberra.edu.au/Security/Accou...,Canberra,Australia,Desktop,Chrome,Maintenance
freq,12334,27,12334,201,1102,993,2289,1242,4119,10003,11653,8129,4005


In [263]:
import datetime as dt
dt.datetime.strptime(iis_logs_df.date.loc[1],'%Y-%m-%d')

#iis_logs_df.date.loc[1]

datetime.datetime(2018, 10, 10, 0, 0)

In [228]:
iis_logs_df.isna().sum()

date                 0
time                 0
server-ip            0
cs-uri-query      6824
server-port          0
cs-username       5059
client-ip            0
cs(User-Agent)     196
cs(Referer)       3128
sc-status            0
sc-substatus         0
time-taken(ms)       0
client-city        126
client-country       0
client-device        0
client-browser       0
client-webPage    4479
dtype: int64

In [224]:


iis_logs_df['cs(Referer)'].apply(lambda x: GetWebPageSection(x) if type(x) != float else np.nan)

#type(iis_logs_df['cs(Referer)'][0])


0                NaN
1                NaN
2                NaN
3                NaN
4           Security
5           Security
6           Security
7           Security
8           Security
9           Security
10          Security
11          Security
12          Security
13          Security
14          Security
15          Security
16          Security
17          Security
18          Security
19          Security
20          Security
21             Admin
22               NaN
23               NaN
24               NaN
25               NaN
26          Security
27               NaN
28               NaN
29               NaN
            ...     
12304            NaN
12305           None
12306           None
12307           None
12308           None
12309           None
12310           None
12311           None
12312           None
12313           None
12314           None
12315           None
12316        Content
12317           None
12318           None
12319        Content
12320        

In [202]:
iis_logs_df['cs(Referer)'].unique()

array([nan,
       'https://inplace.canberra.edu.au/Security/Account.mvc/LogOnFederation?ReturnUrl=%2f',
       'https://inplace.canberra.edu.au/Admin/Theme.mvc/RenderDesktopCss?v=3.8.3.20',
       'https://inplace.canberra.edu.au/Maintenance/Student.mvc/Detail/11299',
       'https://inplace.canberra.edu.au/Maintenance/Student.mvc',
       'https://inplace.canberra.edu.au/Maintenance/Student.mvc/Detail/5355',
       'http://www.baidu.com/s?wd=circlea2j',
       'https://inplace.canberra.edu.au/Security/Account.mvc/LogOnFederation?ReturnUrl=%2F',
       'http://inplace.canberra.edu.au/Content/Styles/global-inplace?v=O8J2N9ILV7RGEQiWyNG4rrE-Ksx2ophGcnZObOuKJ7M1',
       'https://www.canberra.edu.au/myuc-u/study-tools/email',
       'https://inplace.canberra.edu.au/default.aspx',
       'https://inplace.canberra.edu.au/Content/Styles/global-jquery?v=nYJXCCaDzTgv1vGOoG3pxVHR1NWshvyF85Ho-L2HI9M1',
       'https://inplace.canberra.edu.au/Placement/Placement.mvc/Detail/35367?tabId=tab-schedu